In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! cp drive/My\ Drive/11785/hw4_p1/hw4/tests.py .

In [0]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests import test_prediction, test_generation
from tqdm.notebook import tqdm # TODO: delete thie before submit
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [0]:
# load all that we need
# np.random.seed(0) # TODO: remove this

root = "drive/My Drive/11785/hw4_p1/"
dataset = np.load(root + 'dataset/wiki.train.npy', allow_pickle=True)
fixtures_pred = np.load(root + 'fixtures/prediction.npz', allow_pickle=True)  # dev
fixtures_gen = np.load(root + 'fixtures/generation.npy', allow_pickle=True)  # dev
fixtures_pred_test = np.load(root + 'fixtures/prediction_test.npz', allow_pickle=True)  # test
fixtures_gen_test = np.load(root + 'fixtures/generation_test.npy', allow_pickle=True)  # test
vocab = np.load(root + 'dataset/vocab.npy', allow_pickle=True)

In [0]:
# data loader

class LanguageModelDataLoader():
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        self.seq_len = 200
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.dataset = dataset

    def __iter__(self):
        # concatenate your articles and build into batches
        if self.shuffle:
            np.random.shuffle(self.dataset)
        
        dataset_cat = np.concatenate(self.dataset)
        n_seq = len(dataset_cat) // self.seq_len
        dataset_cat = dataset_cat[:n_seq * self.seq_len]
        data = torch.LongTensor(dataset_cat).view(-1, self.seq_len)
        
        for i in range(0, len(data), self.batch_size):
            batch_data = data[i:i+self.batch_size]
            yield (batch_data[:, :-1], batch_data[:, 1:])


In [0]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.num_layers = 1
        self.bidirectional = False # TODO: may change this
        self.input_size = 128
        self.hidden_size = 128
        self.output_size = vocab_size
        
        self.word_embedding = nn.Embedding(self.vocab_size, self.input_size)
        self.lstm = nn.LSTM(input_size=self.input_size, 
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers,
                            dropout=0,
                            batch_first = True,
                            bidirectional=self.bidirectional)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
        print(self.lstm)
        # TODO: initialization of all weights

    def forward(self, x):
        batch_size = x.size(0)
        
        # x.shape: [batch_size, seq_len]
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        x = self.word_embedding(x)
        # x.shape: [batch_size, seq_len, input_size]
        x, _ = self.lstm(x)
        # x.shape: [batch_size, seq_len, hidden_size]
        x = x.contiguous()
        x = self.decoder(x)
        # x.shape: [batch_size, seq_len, vocab_size]
        return x


In [0]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model.to(device)
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.SGD(self.model.parameters(),
                                         lr=0.001, 
                                         weight_decay=1e-6) # TODO: try NT-ASGD
        self.criterion = torch.nn.NLLLoss() # TODO: use splitCrossEntropy

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        with tqdm(total=10500/self.loader.batch_size) as pbar:
            for batch_num, (inputs, targets) in enumerate(self.loader):
                epoch_loss += self.train_batch(inputs, targets)
                pbar.update(1)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = self.model(inputs)        
        
        loss = self.criterion(outputs.view(-1, outputs.size(2)), targets.contiguous().view(-1))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self, exp_path=None):
        # TODO: change this back
        if exp_path:
            model_path = os.path.join(exp_path, self.run_id, 'model-{}.pkl'.format(self.epochs))
            torch.save({'state_dict': self.model.state_dict()},
                model_path)
            np.save(os.path.join(exp_path, self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
            np.save(os.path.join(exp_path, self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
            np.save(os.path.join(exp_path, self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
            np.save(os.path.join(exp_path, self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
            with open(os.path.join(exp_path, self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
                fw.write(self.generated[-1])
            with open(os.path.join(exp_path, self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
                fw.write(self.generated_test[-1])
        else:
            # don't change these
            model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
            torch.save({'state_dict': self.model.state_dict()},
                model_path)
            np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
            np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
            np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
            np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
            with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
                fw.write(self.generated[-1])
            with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
                fw.write(self.generated_test[-1])


In [0]:
class TestLanguageModel:
    def prediction(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        model.to(device)
        inp = torch.LongTensor(inp).to(device)
        outputs = model(inp)
        return outputs[:, -1, :].detach().cpu().numpy()

        
    def generation(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """ 
        model.to(device)
        inp = torch.LongTensor(inp).to(device)
        generated_words = []
                
        embed = model.word_embedding(inp)
        output_lstm, hidden = model.lstm(embed)
        scores = model.decoder(output_lstm[:, -1, :])
        _, current_word = torch.max(scores, dim=1)
        generated_words.append(current_word)
        
        for i in range(forward-1):
            embed = model.word_embedding(current_word).unsqueeze(1)
            output_lstm, hidden = model.lstm(embed, hidden)
            scores = model.decoder(output_lstm[:, -1, :])
            _, current_word = torch.max(scores, dim=1)
            generated_words.append(current_word)
        
        predicts = torch.stack(generated_words).permute(1, 0)
        return predicts.detach().cpu().numpy()
        

In [0]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 5
BATCH_SIZE = 40


In [0]:
run_id = str(int(time.time()))
exp_path = root + "hw4/experiments"
if not os.path.exists(exp_path):
    os.mkdir(exp_path)
os.mkdir(exp_path + '/%s' % run_id)
print("Saving models, predictions, and generated words to " + exp_path + run_id)

In [0]:
model = LanguageModel(len(vocab))
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

In [0]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save(exp_path) # TODO: change this back
    

In [0]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [0]:
# see generated output
print (trainer.generated[-1]) # get last generated output

### DEBUG

In [0]:
dataset_cat = np.concatenate(dataset)
n_seq = len(dataset_cat) // 200
dataset_cat = dataset_cat[:n_seq * 200]
data = torch.LongTensor(dataset_cat).view(-1, 200)

for batch_num, (inputs, targets) in enumerate(loader):
    break
outputs = model(inputs)